In [1]:
import sys
# append the path of the parent directory
sys.path.append("..")

import torch
import torch.nn as nn
import numpy as np
import random

from model import UNet
from dataset import Debris_State_Pairs_Dataset

from train import Trainer

from torch.optim import Adam
from torch.utils.data import DataLoader, Subset, random_split


cuda


In [2]:
def set_seed(seed_value):
    """Set seed for reproducibility."""
    np.random.seed(seed_value)  # NumPy
    torch.manual_seed(seed_value)  # PyTorch
    random.seed(seed_value)  # Python's built-in random module
    
    # minimise non-deterministic GPU behaviour
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Usage
set_seed(42) 

In [ ]:
# Parameters

root_dir = 'C:\\Users\\thomas.bush\\repos\\dyna-landslide-surrogate\\data_small'
batch_size = 64
split_proportions = (0.7, 0.15, 0.15)
epochs = 5

In [1]:
# Data set up

# Initialize dataset with scaling
dataset = Debris_State_Pairs_Dataset(root_dir, array_size=128, apply_scaling=True)

# Split dataset into train, validation, and test sets and create dataloaders
train_loader, val_loader, test_loader = dataset.create_dataloaders(split_proportions, batch_size, random_state=42)

# Compute scaling factors based on the train dataset, but only if scaling is applied
if dataset.apply_scaling:
    train_dataset = train_loader.dataset.dataset 
    dataset.compute_scaling_factors()

# Dataset stats (optional)
print(f"Total dataset size: {len(dataset)}")
print(f"Train size: {len(train_loader.dataset)}, Validation size: {len(val_loader.dataset)}, Test size: {len(test_loader.dataset)}")

In [ ]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on device: {device}.")

# TO DO - add multu GPU support here

In [ ]:
# List of models to train
# models = [
#     {'model': Complex_CNN(), 'name': 'complex_cnn'},
#     {'model': UNet(), 'name': 'unet'},
#     {'model': LargeUNet(), 'name': 'large_unet'}
# ]

# models = [
#     {'model': Complex_CNN(), 'name': 'complex_cnn'}
# ]

models = [
    {'model': UNet(in_channels=3, out_channels=2), 'name': 'unet'}
]

# models = [
#     {'model': LargeUNet(), 'name': 'large_unet'}
# ]

In [ ]:
# Train each model
for model_info in models:
    
    # # Check if multiple GPUs are available and wrap the model using nn.DataParallel
    # if torch.cuda.device_count() > 1:
    #     print(f"Let's use {torch.cuda.device_count()} GPUs!")
    #     # This will wrap the model for use with multiple GPUs
    #     model = torch.nn.DataParallel(model)
    
    # Move model to the appropriate device
    model = model_info['model'].to(device)
    
    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = Adam(model.parameters(), lr=1e-3)
    
    # Initialize the trainer
    model_name = model_info['name']
    trainer = Trainer(model, optimizer, criterion, device, model_name=model_name + "_test")
    
    # Train the model
    print(f"Training {model_name}...")
    trainer.train(train_loader, val_loader, epochs=epochs, checkpoint_interval=5)
    
    # Save the trained model state
    torch.save(model.state_dict(), f"{model_name}_state.pth")

    print(f"Finished training {model_name}. Model state saved.")

In [ ]:
trainer.plot_losses()

In [ ]:
trainer.test(test_loader)


In [ ]:
trainer.plot_predictions(test_loader, num_predictions=1)